<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [ ]:
# TODO - Words, words, mere words, no matter from the heart.

In [2]:
from __future__ import print_function
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.datasets import imdb
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import RMSprop

import numpy as np
import random
import sys
import os

max_features = 20000
maxlen = 80
batch_size = 32

In [5]:
data_url = 'https://www.gutenberg.org/files/100/100-0.txt'

In [17]:
import requests

response = requests.get(data_url)
text = response.text

In [18]:
response.encoding = 'utf-8'
text

0\x99s The Complete Works of William\r\nShakespeare, by William Shakespeare\r\n\r\n*** END OF THIS PROJECT GUTENBERG EBOOK THE COMPLETE WORKS OF WILLIAM\r\nSHAKESPEARE ***\r\n\r\n***** This file should be named 100-0.txt or 100-0.zip ***** This and\r\nall associated files of various formats will be found in:\r\nhttp://www.gutenberg.org/1/0/100/\r\n\r\nUpdated editions will replace the previous one--the old editions will be\r\nrenamed.\r\n\r\nCreating the works from print editions not protected by U.S. copyright\r\nlaw means that no one owns a United States copyright in these works, so\r\nthe Foundation (and you!) can copy and distribute it in the United\r\nStates without permission and without paying copyright royalties.\r\nSpecial rules, set forth in the General Terms of Use part of this\r\nlicense, apply to copying and distributing Project Gutenberg-tm\r\nelectronic works to protect the PROJECT GUTENBERG-tm concept and\r\ntrademark. Project Gutenberg is a registered trademark, and ma

In [19]:
len(text)

5777367

In [20]:
chars = list(set(text))
char_int = {c:i for i, c in enumerate(chars)}
int_char = {i:c for i, c in enumerate(chars)}

In [21]:
# Create the Sequence Data
maxlen = 40
step = 5

encoded = [char_int[c] for c in text]
sequences = [] # Each element is 40 characters long
next_chars = [] # One element for each sequence

for i in range(0, len(encoded) - maxlen, step):
    sequences.append(encoded[i : i + maxlen])
    next_chars.append(encoded[i + maxlen])

print('sequences:', len(sequences))

sequences: 1155466


In [23]:
# Specify x & y

# x is a 3 dimensional matrix
x = np.zeros((len(sequences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sequences), len(chars)), dtype=np.bool)

for i, sequence in enumerate(sequences):
    for t, char in enumerate(sequence):
        x[i, t, char] = 1
        
    y[i, next_chars[i]] = 1

In [24]:
x.shape, y.shape

((1155466, 40, 114), (1155466, 114))

In [25]:
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

In [26]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [29]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.2, 0.5, 0.7, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence

        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_int[char]] = 1

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = int_char[next_index]
            # print("next_char: " + next_char)

            sentence = sentence[1:] + next_char
            # print("sentence[1:]: " + sentence[1:])

            sys.stdout.write(next_char)
            # sys.stdout.flush() # This is bugging, not adding any spaces
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [28]:
model.fit(x, y,
          batch_size=128,
          epochs=5,
          callbacks=[print_callback])

Train on 1155466 samples
Epoch 1/5
1155456/1155466 [============================>.] - ETA: 0s - loss: 2.1140
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: " this fellow.âWhose graveâs this, si"
 this fellow.âWhose graveâs this, sir mad the prove thefor the may the sing of the porter thebeare the will herthe will the porderand the prisele to the beart of the tothe for the for the may so mand
    And the will the will the wordthe mant of the prownes
  And theprood thecome the beat of a more the made the madse
    And the bear the some the will his willthe beart and her the pray the some
  The wher the sond t
----- diversity: 0.5
----- Generating with seed: " this fellow.âWhose graveâs this, si"
 this fellow.âWhose graveâs this, sit you have the will bit the prate.
   He to me lood of the bothmy to king to his a paine
  And the lord youa bast should of the in with of the rime.
    The that in the and the are the will fard.
    The si

In [ ]:
on_epoch_end(5, _)

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN